<a href="https://colab.research.google.com/github/FatiZaha/BookStoreApp/blob/main/(3)_ML_DL_2024_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction au Machine Learning

Bienvenue à la session pratique de notre premier cours d'apprentissage automatique ! Dans ce notebook, nous allons plonger dans les concepts de régression et de classification, en incorporant diverses techniques et observations pour comprendre leur impact sur la performance des modèles.

Ce notebook est conçu non seulement pour vous guider à travers des implémentations pratiques, mais aussi pour servir de ressource d'apprentissage complète. Vous trouverez des explications détaillées et des exemples pour vous aider à saisir les concepts sous-jacents.

## 1. Configuration <a name="setup"></a>

Tout d'abord, nous allons importer les bibliothèques nécessaires. Ces bibliothèques nous fourniront les outils pour le traitement des données, la visualisation et la modélisation.

In [ ]:
# Importer les bibliothèques essentielles
import numpy as np               # Pour les calculs numériques
import pandas as pd              # Pour la manipulation des données
import matplotlib.pyplot as plt  # Pour la visualisation des données
import seaborn as sns            # Pour des visualisations statistiques avancées

# Ignorer les avertissements pour un affichage plus clair
import warnings
warnings.filterwarnings('ignore')

# Bibliothèques d'apprentissage automatique
from sklearn.linear_model import LinearRegression, LogisticRegression   # Modèles de régression et classification
from sklearn.preprocessing import PolynomialFeatures, StandardScaler    # Pour la création de caractéristiques polynomiales et la mise à l'échelle
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier  # Modèles d'arbres de décision pour régression et classification
from sklearn.model_selection import train_test_split, cross_val_score   # Pour la division des données et la validation croisée
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error,                            # Métriques pour la régression
    accuracy_score, precision_score, recall_score, f1_score,            # Métriques pour la classification
    confusion_matrix, ConfusionMatrixDisplay                            # Pour la matrice de confusion
)

## 2. Chargement et Exploration des Données <a name="data-loading-and-exploration"></a>

### 2.1 Chargement du Jeu de Données pour la Régression

Nous allons utiliser la dataset California Housing de scikit-learn. C'est un ensemble de données de régression contenant des informations sur les logements en Californie, basé sur le recensement de 1990. Il inclut des caractéristiques telles que le revenu médian des ménages, l'âge médian des maisons, le nombre moyen de pièces et de chambres par maison, la population, le nombre moyen d'occupants, ainsi que la latitude et la longitude des quartiers. La variable cible est la valeur médiane des maisons (en centaines de milliers de dollars). Cet ensemble de données est souvent utilisé pour des tâches d'apprentissage supervisé, notamment pour explorer et modéliser des relations entre des caractéristiques socio-économiques et les prix des logements.

In [ ]:
# Charger l'ensemble de données
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

# Convertir en DataFrame pandas
data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['MedHouseVal'] = housing.target

# Afficher les cinq premières lignes
data.head()

**Caractéristiques** :

- MedInc : Revenu médian dans le groupe de blocs.
- HouseAge : Âge médian des maisons dans le groupe de blocs.
- AveRooms : Nombre moyen de pièces par ménage.
- AveBedrms : Nombre moyen de chambres par ménage.
- Population : Population du groupe de blocs.
- AveOccup : Nombre moyen de membres du ménage.
- Latitude : Latitude du groupe de blocs.
- Longitude : Longitude du groupe de blocs.

**Cible** :

- MedHouseVal : Valeur médiane des maisons pour les districts de Californie (en $100,000).

Ces informations nous aideront à prédire la valeur médiane des maisons en fonction des caractéristiques fournies.

### 2.2 Chargement du Jeu de Données pour la Classification

Le jeu de données Iris est un ensemble de données d’apprentissage supervisé contenant 150 échantillons de trois espèces de fleurs d'iris : Setosa, Versicolor et Virginica. Chaque échantillon est décrit par quatre caractéristiques numériques : la longueur et la largeur des sépales, ainsi que la longueur et la largeur des pétales (toutes mesurées en centimètres). La variable cible indique l'espèce à laquelle chaque échantillon appartient. Cet ensemble de données est largement utilisé comme point de départ pour expérimenter des algorithmes de classification supervisée, grâce à sa simplicité et à sa petite taille.

In [ ]:
# Charger le jeu de données Iris
from sklearn.datasets import load_iris
iris = load_iris()

# Convertir en DataFrame pandas
iris_data = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_data['Species'] = iris.target

# Afficher les cinq premières lignes
iris_data.head()

**Explication** :

load_iris() charge le jeu de données Iris.
Nous créons un DataFrame pandas pour faciliter l'exploration et le prétraitement.
iris_data.head() nous donne un aperçu des données.
Comprendre le Jeu de Données Iris

**Caractéristiques** :

sepal length (cm) : Longueur du sépale en centimètres.
sepal width (cm) : Largeur du sépale en centimètres.
petal length (cm) : Longueur du pétale en centimètres.
petal width (cm) : Largeur du pétale en centimètres.

**Cible** :

Species : Classe de l'iris (0: setosa, 1: versicolor, 2: virginica).

Le jeu de données Iris est un classique en apprentissage automatique pour les tâches de classification.

## 3. Prétraitement des Données <a name="data-preprocessing"></a>


Le prétraitement des données est une étape cruciale qui peut affecter significativement la performance du modèle. Il inclut la gestion des valeurs manquantes, l'ingénierie des caractéristiques, la sélection des caractéristiques et la mise à l'échelle.

### 3.1 Gestion des Valeurs Manquantes <a name="handling-missing-values"></a>

Pour le jeu de données California Housing :


In [ ]:
# Check for missing values
missing_values = data.isnull().sum()
print(missing_values)

**Observation** : Pas de valeurs manquantes.

Pour le jeu de données Iris :


In [ ]:
# Vérifier les valeurs manquantes
missing_values_iris = iris_data.isnull().sum()
print("\nValeurs manquantes dans le jeu de données Iris :")
print(missing_values_iris)

**Observation** : Pas de valeurs manquantes.

### 3.2 Ingénierie des Caractéristiques <a name="feature-engineering"></a>


L'ingénierie des caractéristiques consiste à créer de nouvelles caractéristiques à partir des données existantes pour améliorer la performance du modèle.



Pour le jeu de données California Housing :

In [ ]:
# Create new features
data['RoomsPerHousehold'] = data['AveRooms'] / data['AveOccup']
data['BedroomsPerRoom'] = data['AveBedrms'] / data['AveRooms']
data['PopulationPerHousehold'] = data['Population'] / data['AveOccup']

**Explication** :

- RoomsPerHousehold : Le nombre moyen de pièces par ménage, ce qui peut indiquer la taille des logements.
- BedroomsPerRoom : La proportion de chambres par rapport aux pièces, reflétant la taille des chambres ou la disposition des maisons.
- PopulationPerHousehold : Le nombre moyen de personnes par ménage, indiquant la densité d'occupation.

Ces nouvelles caractéristiques peuvent capturer des informations non évidentes dans les données originales.

### 3.3 Sélection des Caractéristiques <a name="feature-selection"></a>


La sélection des caractéristiques vise à choisir les variables les plus pertinentes pour le modèle.


In [ ]:
# Calculer la matrice de corrélation
corr_matrix = data.corr()

# Trier les corrélations avec la variable cible
corr_with_target = corr_matrix['MedHouseVal'].sort_values(ascending=False)
print("Corrélations avec la variable cible :")
print(corr_with_target)

**Observation** :

*Analyse des corrélations

- Revenu médian (MedInc) : La plus forte corrélation positive avec la variable cible (0.688), ce qui indique que le revenu médian est un facteur important influençant la valeur des maisons. Plus le revenu médian est élevé, plus la valeur des maisons tend à être élevée.

- Nombre moyen de pièces (AveRooms) : Corrélation positive mais faible (0.151). Cela suggère que le nombre de pièces par maison a un impact limité sur la valeur des maisons.

- Âge médian des maisons (HouseAge) : Corrélation positive très faible (0.105). L'âge des maisons joue un rôle mineur dans la détermination des prix.

- Nombre moyen d'occupants (AveOccup), population (Population), latitude (Latitude), longitude (Longitude), et nombre moyen de chambres (AveBedrms) : Ces variables ont des corrélations faibles ou négatives avec la valeur des maisons. Par exemple, la latitude (-0.144) indique que les maisons situées plus au nord tendent à avoir des valeurs plus faibles.

*Interprétation

- La caractéristique la plus pertinente pour prédire la valeur des maisons est le revenu médian des ménages (MedInc).
- Les autres caractéristiques montrent des corrélations faibles ou négligeables, mais elles pourraient encore être utiles en combinaison avec d'autres variables dans un modèle d'apprentissage.

Sélection des Caractéristiques Basée sur la Corrélation :

In [ ]:
selected_features = [
    'MedInc', 'Latitude', 'Longitude', 'RoomsPerHousehold',
    'BedroomsPerRoom', 'PopulationPerHousehold', 'HouseAge'
]
X = data[selected_features]
y = data['MedHouseVal']

En sélectionnant les caractéristiques les plus pertinentes, nous simplifions le modèle et réduisons le risque de surajustement.
Cela peut également améliorer la performance et la vitesse d'entraînement du modèle.

### 3.4 Mise à l'Échelle des Caractéristiques <a name="feature-scaling"></a>


La mise à l'échelle des caractéristiques peut impacter les algorithmes qui reposent sur des calculs de distance.

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the features
X_scaled = scaler.fit_transform(X)

**Explication** :

StandardScaler standardise les données en supprimant la moyenne et en les mettant à l'échelle de la variance unitaire.
Cela assure que chaque caractéristique contribue de manière égale au modèle.

De même pour le jeu de données Iris :

In [ ]:
# Séparer les caractéristiques et la cible
X_iris = iris_data.drop('Species', axis=1)
y_iris = iris_data['Species']

# Mettre à l'échelle les caractéristiques
X_iris_scaled = scaler.fit_transform(X_iris)

## 4. Division Entraînement-Test <a name="train-test-split"></a>


Diviser les données en ensembles d'entraînement et de test est essentiel pour évaluer la capacité du modèle à généraliser.

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

**Explication** :

- test_size=0.2 signifie que 20% des données sont réservées pour le test.
- random_state=42 assure la reproductibilité des résultats.

Pour le jeu de données Iris :

In [ ]:
# Diviser les données
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X_iris_scaled, y_iris, test_size=0.2, random_state=42)

## 5. Entraînement et Comparaison des Modèles <a name="model-training-and-comparison"></a>
### 5.1 Effet de la Mise à l'Échelle des Caractéristiques <a name="effect-of-feature-scaling"></a>
Nous allons comparer des modèles de Régression Linéaire entraînés avec et sans mise à l'échelle des caractéristiques.

5.1.1 Régression Linéaire sans Mise à l'Échelle

In [ ]:
# Train without scaling
lr_no_scaling = LinearRegression()
lr_no_scaling.fit(X_train, y_train)

# Predict on test set
y_pred_no_scaling = lr_no_scaling.predict(X_test)

**Explication** :

- Nous utilisons les données sans mise à l'échelle pour voir si cela affecte la performance du modèle.
- La Régression Linéaire est moins sensible à la mise à l'échelle, mais il est intéressant d'observer l'impact.

5.1.2 Régression Linéaire avec Mise à l'Échelle

In [ ]:
# Train with scaling
lr_with_scaling = LinearRegression()
lr_with_scaling.fit(X_train, y_train)

# Predict on test set
y_pred_with_scaling = lr_with_scaling.predict(X_test)

**Note** : Dans ce cas, les deux modèles sont entraînés sur des données mises à l'échelle car la régression linéaire n'est pas affectée par la mise à l'échelle en termes de performance, mais la mise à l'échelle peut améliorer la stabilité numérique.

5.1.3 Comparaison de la Performance

In [ ]:
# Calculate RMSE for both models
rmse_no_scaling = np.sqrt(mean_squared_error(y_test, y_pred_no_scaling))
rmse_with_scaling = np.sqrt(mean_squared_error(y_test, y_pred_with_scaling))

print(f'Linear Regression without Scaling RMSE: {rmse_no_scaling:.4f}')
print(f'Linear Regression with Scaling RMSE: {rmse_with_scaling:.4f}')

Discussion :

- Observation : Les valeurs de RMSE sont similaires.
- Explication : La Régression Linéaire n'est pas sensible à la mise à l'échelle des caractéristiques en termes de prédictions car elle trouve les coefficients optimaux indépendamment des échelles des caractéristiques.

### 5.2 Modèles Linéaires vs Non-Linéaires <a name="linear-vs-non-linear-models"></a>




5.2.1 Régression Polynomiale



In [ ]:
# Générer des caractéristiques polynomiales
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

# Entraîner le modèle
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

# Prédire sur l'ensemble de test
y_pred_poly = poly_model.predict(X_test_poly)

Explication :

- La Régression Polynomiale permet de modéliser des relations non linéaires en ajoutant des puissances des caractéristiques originales.
- degree=2 signifie que nous incluons les termes au carré et les interactions entre les caractéristiques.

5.2.2 Régression par Arbre de Décision

Les arbres de décision peuvent capturer des relations complexes et non linéaires.

In [ ]:
# Entraîner le modèle
dt_model = DecisionTreeRegressor(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred_dt = dt_model.predict(X_test)

Explication :

- Les Arbres de Décision partitionnent l'espace des caractéristiques en régions homogènes.
- max_depth=5 limite la profondeur de l'arbre pour éviter le surajustement.

5.2.3 Comparaison des Modèles

In [ ]:
# Mise à l'échelle des données
X_scaled = scaler.fit_transform(data[selected_features])
y = data['MedHouseVal']

# 2. Division des données
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

# 3. Création et entraînement du modèle
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# 4. Prédiction sur les données de test
y_pred_dt = lr_model.predict(X_test)

# 5. Évaluation du modèle
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_dt))

In [ ]:
# Calculer le RMSE
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_with_scaling))
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))
rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))

print(f'RMSE Régression Linéaire : {rmse_linear:.4f}')
print(f'RMSE Régression Polynomiale : {rmse_poly:.4f}')
print(f'RMSE Arbre de Décision : {rmse_dt:.4f}')

**Observation** :

- Comparez les RMSE pour voir quel modèle performe le mieux.
- Le modèle avec le RMSE le plus bas est considéré comme ayant la meilleure performance.

**Explication** :

- Les modèles non linéaires peuvent capturer des relations complexes dans les données.
- Cependant, ils peuvent aussi surajuster si la complexité du modèle est trop élevée.

## 6. Métriques d'Évaluation <a name="evaluation-metrics"></a>


### 6.1 Comparaison des Modèles en Utilisant les Métriques <a name="comparing-models-using-metrics"></a>

Il est important de choisir des métriques appropriées pour évaluer la performance des modèles.

**Quand Utiliser Chaque Métrique?**

- MSE : Lorsque les grandes erreurs sont particulièrement indésirables.
- MAE : Pour une mesure plus robuste face aux valeurs aberrantes.
- RMSE : Pour interpréter l'erreur dans les mêmes unités que la cible.

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    print(f'{model_name} Performance:')
    print(f'MSE: {mse:.4f}')
    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print('-' * 40)

    return {'Model': model_name, 'MSE': mse, 'MAE': mae, 'RMSE': rmse}

In [ ]:
metrics_lr = evaluate_model(y_test, y_pred_with_scaling, 'Linear Regression')
metrics_poly = evaluate_model(y_test, y_pred_poly, 'Polynomial Regression')
metrics_dt = evaluate_model(y_test, y_pred_dt, 'Decision Tree Regression')

Compiler les Métriques dans un DataFrame

In [ ]:
# Create a DataFrame
metrics_df = pd.DataFrame([metrics_lr, metrics_poly, metrics_dt])
metrics_df.set_index('Model', inplace=True)
metrics_df

## 7. Effet des Valeurs Aberrantes <a name="effect-of-outliers"></a>
Les valeurs aberrantes peuvent affecter significativement la performance du modèle, en particulier avec MSE et RMSE.

### 7.1 Introduction de Valeurs Aberrantes

In [ ]:
# Introduire des valeurs aberrantes
y_train_outliers = y_train.copy()
y_train_outliers.iloc[0:5] = y_train_outliers.iloc[0:5] * 1000  # Gonfler les 5 premières valeurs

# Réentraîner la Régression Linéaire
lr_with_outliers = LinearRegression()
lr_with_outliers.fit(X_train, y_train_outliers)

# Prédire sur l'ensemble de test
y_pred_outliers = lr_with_outliers.predict(X_test)

# Évaluer le modèle
metrics_outliers = evaluate_model(y_test, y_pred_outliers, 'Régression Linéaire avec Valeurs Aberrantes')

### 7.2 Observer l'Impact
Comparaison du MSE et du MAE :

- Le MSE devrait augmenter significativement en raison du terme au carré qui pénalise les grandes erreurs.
- Le MAE peut ne pas augmenter aussi dramatiquement.

Discussion :

- Pourquoi le MSE augmente-t-il plus que le MAE ?
- Quand devrions-nous préférer le MAE au MSE ?

Explication :

- Le MSE est plus sensible aux valeurs aberrantes en raison du carré des erreurs.
- Le MAE fournit une mesure plus robuste en présence de valeurs aberrantes.



## 8. Visualisation <a name="visualization"></a>


La visualisation aide à comprendre les performances et les erreurs du modèle.



In [ ]:
# Plot Actual vs. Predicted
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_with_scaling, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valeur Médiane Réelle des Maisons')
plt.ylabel('Valeur Médiane Prédite des Maisons')
plt.title('Régression Linéaire : Réel vs Prédit')
plt.show()

In [ ]:
# Plot Actual vs. Predicted
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_poly, alpha=0.5, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valeur Médiane Réelle des Maisons')
plt.ylabel('Valeur Médiane Prédite des Maisons')
plt.title('Régression Polynomiale : Réel vs Prédit')
plt.show()

**Observation** :

- Alignement avec la Ligne Rouge : Plus les points sont proches de la ligne, meilleures sont les prédictions.


## 9. Retour à la classification

In [ ]:
# Mapper les étiquettes numériques aux noms des espèces pour faciliter l'interprétation des résultats
species_mapping = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
iris_data['Species'] = iris_data['Species'].map(species_mapping)

In [ ]:
# Entraîner le modèle
log_reg = LogisticRegression()
log_reg.fit(X_train_iris, y_train_iris)

# Prédire sur l'ensemble de test
y_pred_log_reg = log_reg.predict(X_test_iris)

In [ ]:
# Entraîner le modèle
dt_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_clf.fit(X_train_iris, y_train_iris)

# Prédire sur l'ensemble de test
y_pred_dt = dt_clf.predict(X_test_iris)

In [ ]:
def evaluate_classification_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    print(f'Performance du {model_name}:')
    print(f'Exactitude : {accuracy:.4f}')
    print(f'Précision : {precision:.4f}')
    print(f'Rappel : {recall:.4f}')
    print(f'F1-score : {f1:.4f}')
    print('-' * 40)

    return {'Modèle': model_name, 'Exactitude': accuracy, 'Précision': precision, 'Rappel': recall, 'F1-score': f1}

In [ ]:
# Évaluer les Modèles
metrics_log_reg = evaluate_classification_model(y_test_iris, y_pred_log_reg, 'Régression Logistique')
metrics_dt = evaluate_classification_model(y_test_iris, y_pred_dt, 'Arbre de Décision')

In [ ]:
# Créer un DataFrame
metrics_classification_df = pd.DataFrame([metrics_log_reg, metrics_dt])
metrics_classification_df.set_index('Modèle', inplace=True)
metrics_classification_df

L'ensemble de données Iris est connu pour être relativement simple, avec des frontières bien définies entre les classes. Cela rend les modèles capables d'apprendre parfaitement les relations et d'obtenir une précision de 100 %.

In [ ]:
print(f"Nombre de lignes Train : {X_train_iris.shape[0]}")
print(f"Nombre de lignes Test  : {X_test_iris.shape[0]}")

In [ ]:
# Afficher la matrice de confusion
ConfusionMatrixDisplay.from_estimator(
    log_reg, X_test_iris, y_test_iris, display_labels=species_mapping.values())
plt.title('Matrice de Confusion - Régression Logistique')
plt.show()

In [ ]:
# Afficher la matrice de confusion
ConfusionMatrixDisplay.from_estimator(
    dt_clf, X_test_iris, y_test_iris, display_labels=species_mapping.values())
plt.title('Matrice de Confusion - Arbre de Décision')
plt.show()

## 10. Conclusion <a name="conclusion"></a>

Dans cette session pratique, nous avons :

- Exploré les concepts de régression et de classification.
- Effectué un prétraitement complet des données.
- Entraîné et comparé plusieurs modèles pour la régression et la classification.
- Compris et utilisé différentes métriques d'évaluation.
- Examiné l'effet des valeurs aberrantes sur la performance des modèles de régression.
- Visualisé les résultats pour une meilleure interprétation.
